In [48]:
pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ----------------------- ---------------- 0.8/1.3 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 4.5 MB/s  0:00:00
   ---------------------------------------- 0.0/14.2 MB ? eta -:--:--
   -- ------------------------------------- 1.0/14.2 MB 5.0 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.2 MB 5.2 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.2 MB 5.1 MB/s eta 0:00:03
   ----------- ---------------------------- 4.2/14.2 MB 5.0 MB/s eta 0:00:02
   --------------- ------------------------ 5.5/14.2 MB 5.2 MB/s eta 0:00:02
   ------------------ --------------------- 6.6/14.2 MB 5.2 MB/s eta 0:00:02
   --------------------- ------------------ 7.6/14.2 MB 5.2 MB/s eta 0:00:02
   --------------------

  You can safely remove it manually.


In [4]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install ipywidgets

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.2 MB 3.9 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 3.7 MB/s  0:00:00

   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
import textwrap
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions

In [2]:
from sentence_transformers import SentenceTransformer

In [4]:
import google.generativeai as genai

In [6]:
# Load variables from .env
load_dotenv()

True

In [7]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
models = genai.GenerativeModel("gemini-2.5-flash")

In [9]:
from datetime import datetime
today = datetime.now().strftime("%B %d, %Y")

In [ ]:
# List available models
# for m in genai.list_models():
#     print(m.name, "supports:", m.supported_generation_methods)

In [ ]:
# response = models.generate_content("Write me a motivational quote about data science.")
# print(response.text)


In [10]:
# ---------------- SETTINGS ----------------
CHROMA_DB_PATH = Path("..") / "output" / "chroma_store"
COLLECTION_NAME = "card_docs"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
N_RESULTS = 5
TEXT_WIDTH = 80
# ------------------------------------------

In [11]:
# Connect to Chroma
client = chromadb.PersistentClient(path=str(CHROMA_DB_PATH))
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL_NAME
)
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn
)

In [15]:
def query_and_summarize(question: str, n_results: int = N_RESULTS, width: int = TEXT_WIDTH):
    """
    Query Chroma, return top chunks with metadata, wrapped text, scores,
    and generate a concise summary using OpenAI LLM.
    """
    # query_embedding = embedding_fn.embed_query(question)

    # # Ensure it is a list of floats
    # if hasattr(query_embedding, "tolist"):  # e.g., numpy array
    #     query_embedding = query_embedding.tolist()

    results = collection.query(
        query_texts=[question],
        # query_embeddings=[query_embedding],
        n_results=n_results,
        include=["metadatas", "documents", "distances"]
    )

    # Wrap text and show relevance scores
    combined_text = ""
    for idx, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][idx]
        score = 1 - results['distances'][0][idx]  # higher = more relevant
        header = (
            f"[Card: {metadata['card']} | Date: {metadata['date']} | "
            f"File: {metadata['filename']} | Page: {metadata['page']} | "
            f"Chunk: {metadata['chunk_index']} | Score: {score:.3f}]"
        )
        wrapped_text = textwrap.fill(doc, width=width)
        combined_text += f"{header}\n{wrapped_text}\n\n"
    
    # Define your prompt
    summary_prompt = (
        f"Today's date is {today}.\n"
        f"Use this date as context when answering.\n\n"
        f"Summarize the following information to answer the question:\n\n"
        f"Question: {question}\n\n"
        f"{combined_text}\n\n"
        f"Provide a short, clear recommendation if applicable. Also point it as bullet points.\n"
        f"Not too many words"
        f"If multiple cards are mentioned, order them based on relevance and keep it concise"
        )

    # Create the model (Gemini 2.5 Flash)
    model = genai.GenerativeModel("gemini-2.5-flash")

    # Generate response
    response = model.generate_content(
        [
            {"role": "model", "parts": "You summarize credit card benefits clearly and concisely."},
            {"role": "user", "parts": summary_prompt}
        ],
        generation_config=genai.types.GenerationConfig(
            max_output_tokens=300,
            temperature=0.7
        )
    )

    # Extract text
    summary = response.text.strip()
        
    return combined_text, summary

In [16]:
if __name__ == "__main__":
    question = "Which card should I use for groceries to get the best cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the information provided, here's which card you should use for groceries to get the best cashback on October 01, 2025:

*   **Amex:** Earns **3% cash back** on the first $6,000 of eligible purchases at U.S. supermarkets in a calendar year (then 1%). This excludes superstores, convenience stores, warehouse clubs, and meal-kit delivery services.
*   **Apple Card:** Earns **2% Daily Cash** back when you use Apple Card with Apple Pay for grocery purchases. If Apple Pay is not accepted, you'll earn 1% Daily Cash.
*   **Discover:** Earns **1% cash back** on groceries. The 5% cash back category for Grocery Stores and Wholesale Clubs for Discover was from April 1 to June 30, 2025, which has already passed.

**Recommendation:**

*   Use your **Amex card** for groceries to get the best cashback of **3%** (up to $6,000 in eligible purchases annually at U.S. supermarkets).


In [25]:
if __name__ == "__main__":
    question = "Which card should I use for groceries to get the best cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the information provided for October 01, 2025:

*   **Amex:** Earns 3% cash back on the first $6,000 of eligible purchases at U.S. supermarkets in a calendar year (then 1%).
*   **Discover:** The 5% cash back category for Grocery Stores and Wholesale Clubs ended on June 30, 2025. Therefore, you would earn 1% cash back on groceries with Discover.
*   **Apple Card:** Offers up to 3% cash back, but it's typically 3% on Apple purchases and select merchants (like Nike and Walgreens when using Apple Pay), 2% with Apple Pay generally, and 1% with the physical card. Groceries are not explicitly mentioned as a 3% category.

**Recommendation:**

*   **Amex** will give you the best cashback for groceries at 3% (up to $6,000 annually).


In [17]:
if __name__ == "__main__":
    question = "Which card should I use for Walmart to get the best cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For the best cashback at Walmart:

*   **Amex:** Use for online Walmart purchases (Walmart.com) to earn **3% cash back** (on the first $6,000 in a calendar year, then 1%). For in-store Walmart purchases, you will earn 1% cash back.
*   **Discover:** For in-store Walmart purchases, you will earn **1% cash back**. The 5% cashback categories mentioned are not active for Walmart during October 2025.
*   **Apple Card:** The specific cashback rate for Walmart is not clearly stated.


In [18]:
if __name__ == "__main__":
    question = "Which card should I use for Kohl's to get the best cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For the best cashback at Kohl's, you should use the:

*   **Kohl’s Rewards® Visa:** Earns 7.5% back at Kohl's stores or Kohls.com.


In [19]:
if __name__ == "__main__":
    question = "Which card should I use for Walgreens to get the best cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For the best cashback at Walgreens:

*   **Apple Card:** Get 3% Daily Cash when using Apple Card with Apple Pay at Walgreens.
    *   Exclusions apply: Not available at Sprint Express, independent health service providers (Walgreens Optical, Walgreens Hearing), or for orders made on third-party delivery apps.
*   **Amex Card:** Walgreens would likely earn 1% cashback as it's not considered a supermarket by Amex.
*   **Citi Card:** No information provided regarding cashback at Walgreens.


In [20]:
if __name__ == "__main__":
    question = "I am buying bananas at a local grocery store, which card to use?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For buying bananas at a local grocery store:

*   **Amex:** Earns 3% cash back on the first $6,000 of eligible purchases at U.S. supermarkets in a calendar year (then 1%).
*   **Apple Card:** Offers unlimited Daily Cash back on all purchases (typically 1% with the physical card, potentially more for specific merchants or with Apple Pay, but not specified for general groceries).

**Recommendation:**

*   **Amex** for 3% cash back on supermarket purchases.


In [21]:
if __name__ == "__main__":
    question = "I am booking a Allegiant flight from SFO TO DFW, which card to use?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For your Allegiant flight from SFO to DFW, you should use the **BOA Allegiant Allways Rewards Visa® card**.

Here's why:

*   **Earn 3 points per $1 spent on Allegiant purchases** (including airfare).
*   **Complimentary priority check-in and priority boarding** for you and everyone on your itinerary.
*   **One Free Beverage** every time you fly Allegiant.


In [22]:
if __name__ == "__main__":
    question = "I am booking a Delta flight from SFO TO DFW, which card to use?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the provided information, there is no specific benefit mentioned for using an Amex card for booking a Delta flight. The excerpts cover entertainment tickets, Send & Split features, Disney streaming credits, and general card terms, but not airline travel rewards or benefits.


In [23]:
if __name__ == "__main__":
    question = "I am buying bananas from Costco, which card to use?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the information provided for October 01, 2025:

*   **Costco Anywhere Visa® Card by Citi:** Earns 2% cash back on all other purchases from Costco and Costco.com.

**Recommendation:**

*   Use the **Costco Anywhere Visa® Card by Citi** to earn 2% cash back on your banana purchase at Costco.


In [24]:
if __name__ == "__main__":
    question = "I am buying bananas, where to buy Costco or Walmart ?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the credit card benefits:

*   **Costco (with Citi Costco Anywhere Visa Card):** You will earn **2% cash back** on your banana purchase.
*   **Walmart (with Discover Card):** Purchases at Walmart are **not eligible** for Discover's 5% grocery store cashback. For the current quarter (October 1 - December 31, 2025), Discover's 5% categories are Amazon.com and Drug Stores, which do not apply to bananas. You would likely earn the standard 1% cash back.

**Recommendation:**

*   Buy bananas at **Costco** using your **Citi Costco Anywhere Visa Card** to earn 2% cash back.


In [26]:
if __name__ == "__main__":
    question = "Where should I buy milk for maximum cashback?"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

For maximum cashback on October 01, 2025:

*   **Discover Card:**
    *   Earn 5% cash back on milk purchased through **Amazon Fresh** (part of the Amazon.com category for Q4 2025, from 10/1/25 - 12/31/25). Remember to activate this quarterly bonus.
*   **Amex Card:**
    *   Earn 3% cash back at **U.S. supermarkets** (excluding superstores, convenience stores, and warehouse clubs) on the first $6,000 in eligible purchases per calendar year.


In [28]:
if __name__ == "__main__":
    question = "Which cards have trip delay insurance benefits?, look into all cards"
    combined_text, summary = query_and_summarize(question, 5, 300)
    print(summary)

Based on the information provided, none of the listed cards explicitly mention trip delay insurance benefits.

*   The Amex documents discuss benefits like Car Rental Loss and Damage Insurance and Purchase Protection, but not trip delay insurance.
*   The BOA Allegiant document focuses on travel rewards, priority services, and a buy-one-get-one airfare benefit, but does not mention trip delay insurance.
